In [ ]:
from importlib import reload
from ipynb.fs.full.notebook_viewer import Viewer, Manager, load_wimage
from script_util import root, fname2id, id2fname, split_dump, search_dump, Filter, prepare, tqdm_load, id2, cmap, rcParams
import pickle
from pickle import Unpickler
from pprint import pprint
from dotdict import dotdict
from functools import reduce

from typing import *
import matplotlib.pyplot as plt
import random
import PIL
from pathlib import Path
import json
from collections import defaultdict
import subprocess
from subprocess import PIPE
from tqdm.notebook import tqdm
from time import time
from functools import lru_cache
import numpy as np
from PIL import Image, ImageDraw
import imagesize
from math import atan2, degrees
import plotly.express as px
from itertools import combinations

plt.rcParams.update(rcParams)
eps = np.finfo(float).eps

In [ ]:
readable_ori =  set((root/"danbooru2020/dump_readable_ori/0").read_text().strip().split())
readable_originalwhite =  set((root/"danbooru2020/dump_readable_originalwhite/0").read_text().strip().split())
missing2x = {"4005075","685111","3554125","3004206","1783239","528288","1717363","4268380","750472","1106509","2072514","2072517","3185545","4039613","3051636","4089762","800782","2515799","1195810","4260855"}
missingsolov2 = {'1084454','2334893','2829693','2832773','2945896','3608576','3657998','3701002','3822349','4054580'}
missinggrav = {'3634238','3634244'}


fire_eggs = dict()
for k in "duplicates is_deleted no_humans not-image photo text_only_page".split():
    fire_eggs[k] = set((root/f"danbooru2020/Danbooru2019/reduce/{k}.txt").read_text().strip().split())
    
ge512 =  set((root/"danbooru2020/ge512.ids").read_text().strip().split())
ge1024 =  set((root/"danbooru2020/ge1024.ids").read_text().strip().split())

i = 2
if not "ids" in globals():
    ids = tqdm_load(root/f"danbooru2020/{i}_id.pkl")
if not "tags" in globals():
    tags = tqdm_load(root/f"danbooru2020/{i}_tag.pkl")

In [ ]:
@lru_cache(maxsize=None)
def t(s):
    if "|" in s:
        retval = reduce(set.__or__, map(tags.get, s.split("|")))
    else:
        retval = reduce(set.__and__, map(tags.get, s.split("&")))
    return retval

In [ ]:
face_one = set((root/"danbooru2020/dump_face_one/0").read_text().strip().split())
instance_one = set((root/"danbooru2020/dump_instance_one/0").read_text().strip().split())

In [ ]:
fl = Filter()
fl["faceinstance"] = (face_one & instance_one) - {'3634238','3634244'}
fl["censored"]     = fl[-1]-tags["censored"]
fl["safe"]         = {_id for _id in fl["censored"] if ids[_id]["rating"] == "s"}
exc1 = t('|'.join([
    "zoom_layer",
    "back|ass|from_behind|from_above|bent_over|standing_split",
    "car|bicycle|translation_request|mystery_skulls",
    "fox_tail|otter_costume|sonic_the_hedgehog",
    "floating_head|cyclops|fat|electric_guitar",
]))|reduce(set.__or__, fire_eggs.values())
fl["out"]          = fl["safe"]-exc1

In [ ]:
samesize = defaultdict(list)
for _id in tqdm(fl["out"]):
    fname = id2.originalwhite(_id)
#    a = tuple(map(int, map(ids[_id].get, ['image_width', 'image_height'])))
    b = imagesize.get(fname)
#    assert a == b
    samesize[b].append(_id)

In [ ]:
s = 0
for k, v in samesize.items():
    s += len(v)*(len(v)-1)//2
print(s)

## ok: RGB空間でのMSE
この時点ではtagsから画像サイズを取っているため実際の画像サイズと不整合がある場合は65536となる

In [ ]:
split_dump(["\t".join(v) for v in samesize.values()        len(v) > 10 ], "samesize"  , N= 1)
split_dump(["\t".join(v) for v in samesize.values() if 1 < len(v) <= 10], "samesize10", N=-1)

In [ ]:
split_dump(
    ["\t".join(c) for c in combinations((root/"danbooru2020/dump_samesize/10").read_text().strip().split(), 2)] + 
    ["\t".join(c) for c in combinations((root/"danbooru2020/dump_samesize/35").read_text().strip().split(), 2)]
,"samesize100")

In [ ]:
fnames = [id2fname(f"{fname2id(f1)}_{fname2id(f2)}", prefix="issame", bucket=lambda _:".") for m,f1,f2 in ok if m < 30]
split_dump([f"-resize 512x512 -background white -gravity center -extent 512x512 {fname}" for fname in fnames], "samesize_resize")

In [ ]:
results = list((root/"danbooru2020/samesize").glob("*"))

In [ ]:
# RGB空間でのMSE
THR = float("inf")
ok = list()
for i, fname in enumerate(tqdm(results)):
    with fname.open("r") as f:
        for line in f:
            m, f1, f2 = line.strip().split()
            if not fname2id(f1) in fl["out"]:
                continue
            if not fname2id(f2) in fl["out"]:
                continue
            m = float(m)
            f12name = id2fname(f"{fname2id(f1)}_{fname2id(f2)}", prefix="issame", bucket=lambda _:".")
            if THR < m:
                break # find ./ -type f|grep -v "_"| xargs -i sort -n {} -o {} 
            else:
                ok.append((m, f12name))
ok.sort()
print(len(ok), ok[0][0], ok[-1][0])

## ok_fix: インスタンスをマスクしてRGB空間でMSE
実際のファイルから画像サイズを取るようにした。人物の差分に注目するため背景のピクセルを計算から除外することにした。

In [ ]:
fnames_fix = [id2fname(f"{fname2id(f1)}_{fname2id(f2)}", prefix="issame", bucket=lambda _:".") for m,f1,f2 in ok_fix if -1 < m <= 221]

In [ ]:
def append_option(dname):
    _id = fname2id(dname)
    f1name = id2.originalwhite(_id.split("_")[0])
    f2name = id2.originalwhite(_id.split("_")[1])
    return f"convert +append {f1name} {f2name} -resize 256x384 -background white -gravity center -extent 256x384 {dname}"
(root/"danbooru2020/issame_fix").mkdir(exist_ok=True)

In [ ]:
split_dump(list(map(append_option, fnames_fix)), "option_issame_fix")

In [ ]:
results_fix = list((root/"danbooru2020/samesize_fix").glob("*"))

In [ ]:
# RGB空間でインスタンスをマスクしてMSE
#THR_FIX = 221
THR_FIX = float("inf")
ok_fix = list()
for i, fname in enumerate(tqdm(results_fix)):
    with fname.open("r") as f:
        for line in f:
            m, f1, f2 = line.strip().split()
            if not fname2id(f1) in fl["out"]:
                continue
            if not fname2id(f2) in fl["out"]:
                continue
            m = float(m)
            f12name = id2fname(f"{fname2id(f1)}_{fname2id(f2)}", prefix="issame", bucket=lambda _:".")
            if THR_FIX < m:
                break # find ./ -type f|grep -v "_"| xargs -i sort -n {} -o {} 
            else:
                ok_fix.append((m, f12name))
ok_fix.sort()
print(len(ok_fix), ok_fix[0][0], ok_fix[-1][0])

In [ ]:
fig = px.histogram([a[0] for a in ok_fix])
fig.show()

In [ ]:
#viewer_fix.bag |= {id2fname(f"{fname2id(f1)}_{fname2id(f2)}", prefix="issame", bucket=lambda _:".") for m,f1,f2 in ok_fix if -1 < m <= 70}

In [ ]:
viewer_fix = Viewer(fl["0"], key = "nosort", name ="reduce_duplicates_fix", N=13*8-1)
viewer_fix

In [ ]:
fnames = [ for m,f1,f2 in ok if m < 30]
print(len(fnames))

## ok_masklab: インスタンスをマスクしてLab空間上でMSE
fl["faceinstance"]は巨大すぎて見切れないので、fl["out"]に対象を絞った。RGB空間よりLab空間上での距離の方が人間の色覚に即した距離が出るため、補助的に採用した。

In [ ]:
results_masklab = list((root/"danbooru2020/masklab").glob("*"))

In [ ]:
ok_masklab_all = list()
for i, fname in enumerate(tqdm(results_masklab)):
    with fname.open("r") as f:
        for line in f:
            m, f12name = line.strip().split()
            fid1, fid2 = fname2id(f12name).split("_")
            if not fid1 in fl["out"]:
                continue
            if not fid2 in fl["out"]:
                continue
            m = float(m)
            ok_masklab_all.append((m, f12name))
ok_masklab_all.sort()
print(len(ok_masklab_all), ok_masklab_all[0][0], ok_masklab_all[-1][0])

In [ ]:
Manager([f12name for m,f12name in ok_fix], key = "nosort", name ="reduce_duplicates_fix_out")

In [ ]:
bag = Manager.MANAGED["reduce_duplicates_fix_out"].bag
bin = 10
f=int

In [ ]:
cnt = [0]*(1+f(max(ok_fix)[0])//bin)
yes = [0]*len(cnt)
for m, f12name in tqdm(ok_fix):
    cnt[f(m)//bin] += 1
    if f12name in bag:
        yes[f(m)//bin] += 1

In [ ]:
data1 = [100*y/(c+eps) for c, y in zip(cnt, yes)]
data2 = cnt

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

X = [bin*i for i in range(len(data1))]
ax1.set_ylabel('見つかった重複・差分画像の割合', color=cmap(3)) 
ax1.plot(X, data1, color=cmap(3))
ax1.tick_params(axis='y', labelcolor=cmap(3))

ax2.set_ylabel('このMSEの値を取る2枚の画像組の件数', color=cmap(2))
ax2.plot(X, data2, color=cmap(2))
ax2.tick_params(axis='y', labelcolor=cmap(2))

ax1.set_xlabel('MSE（人の写った領域のみに注目して計算）')
plt.title("サイズの等しい二枚の画像を目視で重複除去した結果")

fig.tight_layout()# otherwise the right y-label is slightly clipped
plt.show()